# Create Accessibility Layers

## Objectives:
1. Create catchment areas for each PoI based on least cost distance in minutes for different travel mode( walking or walking + motorized )
2. Create least path distance in minutes from settlement locations to the closest PoI
3. Calculate everage distance in minutes per settlement to the closest PoI based on travel mode ( walking or walking + motorized )
4. Summarize popuation by the cathcment areas
5. Summarize population in 30min time interval by catchment areas

## Input Layers:
1. Friction layer in minutes for walk travel type
2. Friction layer in minutes for mix travel  travel ( walking or walking + motorized ) type
3. Popualtion raster layer. WorldPop constrainted popualiton layer shoud be used. 
4. Settlement extent
5. Point of interest (health facilities,schools etc)

## Outputs :
1. Cost distance surface in minutes
2. Catchent areas of each PoI with popualtion summary
3. Catchent areas of each PoI ( boundaries are smooth) 
4. Least cost path from settlement locations to closest PoI
5. Least cost path from settlement locations to closest PoI (lines are smooth)
6. Settlement extens with everage travel time in minutes to closest PoI

## Process

### Import require libraries

In [16]:
import os
import pandas as pd
import arcpy
import re
import arcpy.cartography as CA
arcpy.env.overwriteOutput = True
import unidecode

     

In [115]:
##install libraries if you have not done before
#! pip install unidecode

### Initialize Variables

In [26]:
country_iso="sle"           # three letter country iso code e.g. zmb for Zombia
poi_type=  "schools"        # type of point of interest such as "health_facility"      !! no space between names !!

##=============================================================================================##
travel_type= "walk"          # options: walk or mix( walk+motorized)
run_by="all"            # if you want to create accesibiliy layers for each poi type "category", otherwise type "all" 
poi_type_col="sch_type"      # name of the field/column for poi type

###================================ input layers ==============================================###

# path to poi such as health facilities or schools
path_to_poi=r""
# friction surface for walking. It is created by "create_friction_surface" script
path_to_friction_walk=r""
# friction surface for motorized.It is created by "create_friction_surface" script
path_to_friction_mix=r""
#path to popualtion raster
path_to_pop_raster=r"
# path to settlement extent
path_to_sett_extent=r""
### important: Settlement extent layers (bua, saa , hamlets) need to be merged. 
               # if settlement extent layers are merged "path_to_sett_extent" needs to be initialized to merged layer
               # if settlement extent layers are not merged "path_to_sett_extent" needs to be initialized to the   
               #  gdb that have settlement extent layers,and "Merge settlement extent" section needs to be run

            # path to the directory that you want to save outputs
output_path=r""

###==================== select friction layer based on travel type============================###

if travel_type=="walk":
    friction_layer= path_to_friction_walk
if travel_type=="mix":
    friction_layer= path_to_friction_mix
            
###================================ output workspace ==========================================###


# create output directory
if not  os.path.exists (os.path.join(output_path,"output_"+poi_type+"_"+run_by+"_"+travel_type)):
    os.mkdir(os.path.join(output_path,"output_"+poi_type+"_"+run_by+"_"+travel_type))
output_loc=os.path.join(output_path,"output_"+poi_type+"_"+run_by+"_"+travel_type)

# create output gdb
if  not arcpy.Exists(os.path.join(output_loc,"output.gdb")):
    arcpy.CreateFileGDB_management(output_loc,"output.gdb")
output_gdb=os.path.join(output_loc,"output.gdb")


arcpy.env.workspace = output_gdb


### Preprocessing


#### Merge settlement extent

In [2]:
# ### Run this section if settlement extent layer has a seperate layer for each settlement class

# bua=os.path.join(path_to_sett_extent,"bua")
# ssa=os.path.join(path_to_sett_extent,"ssa")
# hamlet=os.path.join(path_to_sett_extent,"hamlet")
# arcpy.Merge_menagament([bua, ssa, hamlte], "sett_extent_merged")
# path_to_sett_extent=os.path.join(output_gdb,"sett_extent_merged")

### Process

### Functions

In [27]:

def project_raster(raster_to_be_projected, reference_raster):
    arcpy.env.snapRaster  =reference_raster
    arcpy.env.extent      =reference_raster
    arcpy.env.cellAlignment = "ALIGN_WITH_PROCESSING_EXTENT"
    arcpy.env.cellSize = reference_raster
    arcpy.env.overwriteOutput = True 
    arcpy.env.outputCoordinateSystem = arcpy.Describe(reference_raster).spatialReference
    prj= arcpy.Describe(reference_raster).spatialReference
    arcpy.ProjectRaster_management(raster_to_be_projected,"pop_proj",prj)


def spatial_join(target, join, fieldname, out_name):
    fieldmappings = arcpy.FieldMappings()
    fieldmappings.addTable(target)
    fieldmappings.addTable(join)
    fieldIndex_ = fieldmappings.findFieldMapIndex("grid_code")
    fieldmap = fieldmappings.getFieldMap(fieldIndex_ )
    field = fieldmap.outputField
    field.name = fieldname
    field.aliasName = fieldname
    fieldmap.outputField = field
    fieldmap.mergeRule = "mean"
    fieldmappings.replaceFieldMap(fieldIndex_ , fieldmap)
    arcpy.SpatialJoin_analysis(target, join,out_name, "#", "#", fieldmappings)

def create_accessibility_layers(point_layer, outname="all"):    
     ##==========================================================================================###
    cost_distance="cost_distance_"+ outname
    cost_allocation="cost_allocation_"+ outname
    catchment_areas="catchments_"+ outname
    catchment_areas_smoothBorders= "catchments_smoothBorders_"+ outname
    sett_path="sett_path_"+ outname
    sett_path_smoothline="sett_path_all_smoothlines_"+ outname
    sett_with_cost="Sett_extent_cost_in_min_"+ outname
    point_layer=arcpy.CopyFeatures_management(point_layer, poi_type+"_"+outname)
    ##==========================================================================================### 
   
    print (f"Craating accessiblility layers for =={country_iso}== for =={poi_type} >> {outname}== with =={travel_type}== travel mode" )
    print(" ")
    print ( "   >>> Creating cost distance in minutes to PoI")
    arcpy.sa.CostDistance(point_layer,friction_layer,"" , "outBkLinkRaster").save(  cost_distance)
    
    print ( "   >>> Creating catcment areas of PoI  based on  minimum cost")
    arcpy.sa.CostAllocation(point_layer,friction_layer,"","","OBJECTID","cost_allocation").save("cost_allocation_temp")
    arcpy.RasterToPolygon_conversion("cost_allocation_temp", "catchment_areas_temp", "NO_SIMPLIFY")
    
    
    print ("   >>> Calculating population of each catchmet area by 30 min dist. interval")     
    arcpy.sa.Con(cost_distance,130, cost_distance, "VALUE>120").save(cost_distance+"_con")
    arcpy.sa.Contour(cost_distance+"_con", cost_distance+"_contour", 30, 0, 1, "CONTOUR_POLYGON")
    arcpy.AddField_management(cost_distance+"_contour", "time_interval", "TEXT")
    arcpy.management.CalculateField(cost_distance+"_contour", "time_interval", 'str(!ContourMin!)+"-"+str(!ContourMax!)', "PYTHON3", '', "TEXT")
    arcpy.Intersect_analysis([cost_distance+"_contour","catchment_areas_temp"], "catchment_areas_int", "ALL")
    arcpy.AddField_management( "catchment_areas_int", "zone_unique", "TEXT")
    arcpy.management.CalculateField( "catchment_areas_int", "zone_unique", 'str(!gridcode!)+"_"+str(!time_interval!)', "PYTHON3", '', "TEXT")
    arcpy.sa.ZonalStatisticsAsTable("catchment_areas_int", "zone_unique", path_to_pop_raster, "zonal_pop_min","DATA", "SUM")
    totpop_min_array = arcpy.da.TableToNumPyArray("zonal_pop_min",["zone_unique","SUM"])
    df=pd.DataFrame(totpop_min_array)
    df[['gridcode','time_interval']] = df[ "zone_unique"].str.split("_",expand=True) 
    df= df.pivot(index='gridcode', columns='time_interval', values="SUM")
    df.rename({"0.0-30.0":"under_30min","30.0-60.0":"30min-60min","60.0-90.0":"60min-90min",
                 "90.0-120.0":"90min-120min","120.0-130.0":"above_120min"}, axis=1, inplace=True)
    df=   df.fillna(0)
    df["totpop_20"]=df["under_30min"]+df["30min-60min"]+df["60min-90min"]+df["90min-120min"]+df["above_120min"]
    df.to_csv(output_loc+"\\"+"catchments_pop_summary.csv")
    pop_summary=os.path.join(output_loc,"catchments_pop_summary.csv")
    
    print ("   >>> Adding population and PoI attribute fields to catchment area layer")  
    arcpy.AddJoin_management("catchment_areas_temp", "gridcode",pop_summary, "gridcode")  
    arcpy.AddJoin_management("catchment_areas_temp", "gridcode", point_layer, "OBJECTID")
    arcpy.CopyFeatures_management("catchment_areas_temp", catchment_areas)      
     
    print ("   >>> Smoothing catchment areas boundarie")
    CA.SimplifyPolygon( catchment_areas,catchment_areas_smoothBorders, "POINT_REMOVE", 50,"","","NO_KEEP")  
                    
    print ("   >>> Creating cost path from settlements to PoI")
    arcpy.sa.CostPath(path_to_sett_extent, cost_distance, "outBkLinkRaster","EACH_ZONE","OBJECTID").save("settlement_path_temp")
    arcpy.RasterToPolyline_conversion("settlement_path_temp", sett_path,"", "", "NO_SIMPLIFY", "Value")
    
    print ("   >>> Smoothing paths")
    CA.SimplifyLine(sett_path,sett_path_smoothline, "POINT_REMOVE", 50,"","NO_KEEP")
    
    print ( "   >>> Calculating everage travel time in minutes to closest PoI...")
    arcpy.RasterToPoint_conversion(cost_distance, "cost_distance_point", "VALUE")
    spatial_join(path_to_sett_extent, "cost_distance_point", "average_cost_min", sett_with_cost)  
    arcpy.DeleteField_management( sett_with_cost,  ["Join_Count", "TARGET_FID", "pointid"])
    
    print ("   >>> Deleting temporary layers...")
    delete_fc=["outBkLinkRaster","cost_allocation_temp", "cost_distance_point","catchment_areas_temp","catchment_areas_int",
               "zonal_pop_min","settlement_path_temp","cost_allocation","cost_distance_all_con","cost_distance_all_contour"]
    for fc in delete_fc:
         arcpy.Delete_management(fc)      
    print ("##====================================================================================================##")    
          


    

### Create Outputs

In [28]:
## project popualtion raster to aling with friction raster
project_raster(path_to_pop_raster,friction_layer)
path_to_pop_raster="pop_proj"

### this section will be executed if run_by set as "all"
if run_by=="all":
    create_accessibility_layers(path_to_poi)
    
##=========================================================================================================##

## this section will be executed if run_by set as "category"
if run_by=="category":
    ## get list of category, but before that forma type
    ## formatting output layers names for each category in PoI layer
    ## arcpy does not create output layers if the layers name contains spatial characters
    category_list=[]
    arcpy.AddField_management(path_to_poi, "type_clean", "TEXT")
    with arcpy.da.UpdateCursor(path_to_poi,[poi_type_col,"type_clean"]) as cursor:
        for row in cursor:
            if row[0] is not None:
                fix_type= unidecode.unidecode(row[0])
                fix_type=re.sub(r'[-|$|.|!|)|(|!|-|_|]',r'',fix_type)
                fix_type=fix_type.strip().replace("  ","_").replace(" ","_")
                row[1]=fix_type
                category_list.append(row[1])
                cursor.updateRow(row)
    category_list=set(category_list)
    
    print (f"There are {len(category_list)} categories in input PoI layer. These are:")
    print (category_list)
    print ("##=========================================================================================================##")

    for category in category_list:
        category_poi=arcpy.Select_analysis(path_to_poi,category,"type_clean = '{}'".format(category))
        create_accessibility_layers(category_poi,outname=category)
        
### delete column that added to input  Poi layer and settlement extent 
arcpy.DeleteField_management(  path_to_poi, "type_clean")


Craating accessiblility layers for ==sle== for ==schools >> all== with ==walk== travel mode
 
   >>> Creating cost distance in minutes to PoI
   >>> Creating catcment areas of PoI  based on  minimum cost
   >>> Calculating population of each catchmet area by 30 min dist. interval
   >>> Adding population and PoI attribute fields to catchment area layer
   >>> Smoothing catchment areas boundarie
   >>> Creating cost path from settlements to PoI
   >>> Smoothing paths
   >>> Calculating everage travel time in minutes to closest PoI...
   >>> Deleting temporary layers...
##====================================================================================================##


<Result 'C:\\Users\\hengin_cidr\\Desktop\\hasim_recovery\\Columbia\\SLE\\SLE_school_catchment_areas\\input_layers\\schools\\D_Senior_Secondary.shp'>